<a href="https://colab.research.google.com/github/plpowerbug/code-eval/blob/main/evaluate_r1_qwen3coder_humaneval_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> # 🤖 HumanEval 测试： r1Qwen3Coder 32B instruct



In [ ]:
!pip install transformers datasets pandas nltk --quiet

In [ ]:
import torch
import pandas as pd
import tempfile
import subprocess
import nltk
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True



```
# 此内容为代码格式
```

 加载  Qwen2.5Coder 模型

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): 



```
#加载 HumanEval 并生成代码

```
# 此内容为代码格式
```


```



In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("openai_humaneval")
samples = dataset["test"]
print(f"共加载 {len(samples)} 道 HumanEval 题目")

README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

共加载 164 道 HumanEval 题目


定义生成 + 测试代码的函数

In [ ]:
# 生成代码
def generate_code(prompt, max_new_tokens=256):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()

# 执行代码并跑 test 部分
def run_test(function_code: str, test_code: str) -> bool:
    with tempfile.NamedTemporaryFile(suffix=".py", mode="w+", delete=False) as f:
        f.write(function_code + "\n\n" + test_code)
        f.flush()
        try:
            subprocess.check_output(["python3", f.name], stderr=subprocess.STDOUT, timeout=5)
            return True
        except subprocess.CalledProcessError:
            return False
        except subprocess.TimeoutExpired:
            return False

In [ ]:
import tempfile
import subprocess
results = []
num_completions = 3  # 每题生成几个解法
subset = samples  # 评估前 5 题，可改 range(10) 评估前10题

for item in subset:
    for i in range(num_completions):
        generated = generate_code(item["prompt"])
        passed = run_test(item["prompt"] + generated, item["test"])
        results.append({
            "task_id": item["task_id"],
            "index": i + 1,
            "passed": passed,
            "completion": generated
        })
        print(f"{item['task_id']} | Try {i+1} | {'✅ PASS' if passed else '❌ FAIL'}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


HumanEval/0 | Try 1 | ❌ FAIL
HumanEval/0 | Try 2 | ❌ FAIL
HumanEval/0 | Try 3 | ✅ PASS
HumanEval/1 | Try 1 | ✅ PASS
HumanEval/1 | Try 2 | ❌ FAIL
HumanEval/1 | Try 3 | ❌ FAIL
HumanEval/2 | Try 1 | ❌ FAIL
HumanEval/2 | Try 2 | ❌ FAIL
HumanEval/2 | Try 3 | ❌ FAIL
HumanEval/3 | Try 1 | ❌ FAIL
HumanEval/3 | Try 2 | ❌ FAIL
HumanEval/3 | Try 3 | ❌ FAIL
HumanEval/4 | Try 1 | ❌ FAIL
HumanEval/4 | Try 2 | ❌ FAIL
HumanEval/4 | Try 3 | ❌ FAIL
HumanEval/5 | Try 1 | ❌ FAIL
HumanEval/5 | Try 2 | ❌ FAIL
HumanEval/5 | Try 3 | ❌ FAIL
HumanEval/6 | Try 1 | ✅ PASS
HumanEval/6 | Try 2 | ❌ FAIL
HumanEval/6 | Try 3 | ❌ FAIL
HumanEval/7 | Try 1 | ✅ PASS
HumanEval/7 | Try 2 | ❌ FAIL
HumanEval/7 | Try 3 | ❌ FAIL
HumanEval/8 | Try 1 | ❌ FAIL
HumanEval/8 | Try 2 | ❌ FAIL
HumanEval/8 | Try 3 | ✅ PASS
HumanEval/9 | Try 1 | ❌ FAIL
HumanEval/9 | Try 2 | ❌ FAIL
HumanEval/9 | Try 3 | ❌ FAIL
HumanEval/10 | Try 1 | ✅ PASS
HumanEval/10 | Try 2 | ✅ PASS
HumanEval/10 | Try 3 | ✅ PASS
HumanEval/11 | Try 1 | ❌ FAIL
HumanEval/